In [ ]:
!pip install cvxpy

In [ ]:
# !pip install gurobipy

In [ ]:
import cvxpy as cp
# import gurobipy as gp
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import NearestCentroid
from scipy.spatial.distance import cdist


In [ ]:
np.random.seed(1)


In [ ]:
dat=load_breast_cancer(return_X_y=True, as_frame=False)

In [ ]:
X,y=dat[0],dat[1]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42) #rs 0,1 pinv also doesnt work

In [ ]:
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [ ]:
clf = LogisticRegression(random_state=0).fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# clf=MLPClassifier(random_state=0,hidden_layer_sizes=(10,),activation='identity').fit(x_train,y_train)

In [ ]:
def create_obj_var(x,n_ft,ft_type):
  vars=[]
  obj_var=[0]*n_ft
  for i in range(n_ft):
    # vars[i]=cp.Variable()
    # print(ft_type[i])
    if ft_type[i]=='ct':
      # print(vars[i])
      v=cp.Variable()
      obj_var[i]=cp.norm(v-x[i],1)/4 #should add denominator
    elif ft_type[i]=='in':
      v=cp.Variable(integer=True)
      obj_var[i]=cp.norm(v-x[i],1)/4
    else:
      v=cp.Variable(boolean=True)
      obj_var[i]=(cp.maximum(1,(v-x[i])/1000))
    vars.append(v)
  # vars=cp.Variable(n_ft)
  return vars,obj_var



In [ ]:
def create_obj_var2(x,n_ft,ft_type,A,c,target):
  vars=[]
  nu=0.5
  obj_var=[0]*n_ft
  if target==0:
    A=A[0]
    c=c[0]
  else:
    A=A[1]
    c=c[1]
  for i in range(n_ft):
    # vars[i]=cp.Variable()
    # print(ft_type[i])
    if ft_type[i]=='ct':
      # print(vars[i])
      v=cp.Variable()
      obj_var[i]=cp.norm(v-x[i],1)/4 #should add denominator
    elif ft_type[i]=='in':
      v=cp.Variable(integer=True)
      obj_var[i]=cp.norm(v-x[i],1)/4
    else:
      v=cp.Variable(boolean=True)
      obj_var[i]=(cp.maximum(1,(v-x[i])/1000))

    vars.append(v)

  for i in range(n_ft):
    obj_var[i]=obj_var[i]++nu*cp.norm(A[i,:]@(vars-c),1)
  # vars=cp.Variable(n_ft)
  return vars,obj_var



In [ ]:
# vars,_=create_obj_var(x_test[0],30,['ct']*30)
# vars

In [ ]:
# p=vars@clf.coef_[0].T
# cp.exp(p)/1+cp.exp(p)

In [ ]:
# np.logaddexp(-1000,x_test[0]@clf.coef_[0]+clf.intercept_[0])

In [ ]:
# p=x_test[0]@clf.coef_[0]+clf.intercept_[0]
# (np.exp(p)/(1+np.exp(p)))
# p

In [ ]:
def check_pred(cf,clf,target,clf_type='linear'): #assume mlp activation is identity
  if clf_type=='linear':
    cf=cp.hstack(cf)
    # p=cp.log_sum_exp(cp.hstack([-1000,cf@clf.coef_[0].T+clf.intercept_[0]]))
    pr=cp.matmul(cf,clf.coef_[0])+clf.intercept_[0]
    # p=cp.exp(p)/(1+cp.exp(p))
    p=cp.sum(cp.exp(cp.abs(pr - cp.log(target))))

  elif clf_type=='mlp':
    pr=cp.hstack(cf)
    for i in range(len(clf.coefs_)):
      pr=cp.matmul(pr,clf.coefs_[i])+clf.intercepts_[i]

    p=cp.sum(cp.exp(cp.abs(pr - cp.log(target))))
  return p,pr



In [ ]:
def get_anchors(x_train,y_train):
  cls1_idx=np.where(y_train==1)
  cls0_idx=np.where(y_train==0)

  cls0=x_train[cls0_idx]
  cls1=x_train[cls1_idx]

  cls0_var=np.linalg.pinv(np.cov(cls0.T))
  cls1_var=np.linalg.pinv(np.cov(cls1.T))

  clf = NearestCentroid()
  clf.fit(x_train,y_train)

  return cls0_var,cls1_var,clf.centroids_



In [ ]:
# import cvxpy as cp
# import numpy as np
# from sklearn.linear_model import LogisticRegression

# # Generate some example data
# np.random.seed(0)  # For reproducibility
# X = np.random.rand(100, 2)  # Example input features
# y = np.random.randint(0, 2, 100)  # Example binary labels (0 or 1)

# # Fit a logistic regression model
# logreg_model = LogisticRegression()
# logreg_model.fit(X, y)

# # Define cvxpy variable for the input vector
# x = cp.Variable(2)

# # Compute the log odds (linear part of logistic regression)
# log_odds = cp.matmul(x, logreg_model.coef_[0]) + logreg_model.intercept_

# # Define the target probability (e.g., close to zero)
# target_probability = 0.3  # Adjust as needed

# # Define the log-sum-exp constraint indirectly
# constraint_lhs = cp.sum(cp.exp(log_odds - cp.log(target_probability)))
# constraint_rhs = len(X)  # Number of data points
# constraint = constraint_lhs <= constraint_rhs

# # Create the cvxpy problem with the constraint
# problem = cp.Problem(cp.Minimize(0), [constraint])

# # Solve the problem
# problem.solve()

# # Access the optimized input vector
# optimal_vector = x.value

# # Evaluate the log odds for the optimal vector
# optimal_log_odds = np.dot(optimal_vector, logreg_model.coef_[0]) + logreg_model.intercept_

# print("Optimal Input Vector:", optimal_vector)
# print("Log Odds for Optimal Vector:", optimal_log_odds)


In [ ]:
# Create constraint.
def optimize(vars,obj_var,clf,x,target):
  # vars.append(cp.Variable())
  # x=x.reshape(1,-1)

  # p=cp.sum(vars*np.transpose(np.array(clf.coef_))) +clf.intercept_[0]
  # print(p)
  # p=1/1+cp.exp(-p)


  p,pr=check_pred(vars,clf,target,'mlp')
  constraints=[p<=1]
  # constraints=[]

  # cls0_var,cls1_var,centroids=get_anchors(x_train,y_train)
  # c1=centroids[1]
  # c0=centroids[0]
  # if target==0:
  #   # obj2=cp.matmul((cp.vstack(vars)-cp.vstack(c0)).T,cp.matmul(cls0_var,(cp.vstack(vars)-cp.vstack(c0))))
  #   obj2=cp.norm(cls0_var@(vars-c0),1)
  # else:
  #   obj2=cp.norm(cls1_var@(vars-c1),1)
  #   # obj2=cp.matmul((cp.vstack(vars)-cp.vstack(c1)).T,cp.matmul(cls1_var,(cp.vstack(vars)-cp.vstack(c1))))
  # Form objective.
  # objective = cp.Minimize(cp.sqrt(cp.sum_squares(cp.vstack(obj_var)))) #
  objective = cp.Minimize(cp.sum(cp.vstack(obj_var)))
  # objective = cp.Minimize(cp.norm(v-x,1))

  # vars.value = numpy.random.randn(30)


  # Form and solve problem.
  prob = cp.Problem(objective,constraints)
  prob.solve(solver=cp.SCS,qcp=True,verbose=True) #cp.SCS, cp.ECOS_BB
  # prob.solve(warm_start=True)
  print("status: {}".format(prob.status))
  return vars

In [ ]:
cf_all=[]
ft_type=['ct']*30
n_ft=30
for k in range(len(x_test)):
  i=x_test[k]
  target=1-y_test[k]
  cls0_var,cls1_var,centroids=get_anchors(x_train,y_train)
  vars,obj_var=create_obj_var2(i,n_ft,ft_type,[cls0_var,cls1_var],centroids,target)
  # vars,obj_var=create_obj_var(i,n_ft,ft_type)
  cf=[]
  target=0.001 if target==0 else 0.999
  vals=optimize(vars,obj_var,clf,i,target)
  for j in vals:
    cf.append(j.value)
  cf_all.append(cf)

NameError: name 'x_test' is not defined

In [ ]:
np.array(cf_all).shape, y_test.shape

((114, 30), (114,))

In [ ]:
def check_validity(cf_all,y_test):
  validity=0
  for i in range(len(cf_all)):
    if clf.predict(np.array(cf_all[i]).reshape(1,-1))==1-y_test[i]:
      validity+=1

  return validity/len(y_test)

In [ ]:
def l1_dist(cf,x_test):
  d=[]
  for i in range(len(x_test)):
    d.append(cdist([x_test[0]],[cf[i]],metric='cityblock'))

  return np.mean(d)


In [ ]:
def maha_dist(cf,x_test,cov):
  d=[]

  for i in range(len(x_test)):
    diff=cf[i]-x_test[i]
    d.append(np.dot(diff.T,np.dot(cov,diff)))
  return np.mean(d)

In [ ]:
dist=l1_dist(cf_all,x_test)

In [ ]:
mdist=maha_dist(cf_all,x_test,np.linalg.pinv(np.cov(x_test.T)))

In [ ]:
def get_stats(x_train,ft_types):
  '''
  should receive the raw data
  '''
  n_ft= x_train.shape[1]
  stats=[]
  for i in range(n_ft):
    s=[]
    if ft_types[i]=='ct':
      s[0]=np.mean(x_train[:,i])
      s[1]=np.var(x_train[:,i])**0.5
    else: #consider only cont and cat currently. Need discrete data also.
      s[0]=np.min(x_train[:,i])
      s[1]=np.max(x_train[:,i])
    stats.append(s)

  return stats


In [ ]:
def gen_random_point(stats,ft_types):
  randx=[]
  for i in range(len(stats)):
    val=0
    if ft_types[i]=='continuous':
      val=np.randn(stats[i][0],stats[i][1],size=None)
    else:
      val=np.random.randint(stats[i][0],stats[i][1])
    randx[i]=val

  return randx



5 fold

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
import time

In [ ]:
l1_dist_all=[]
maha_dist_all=[]
time_all=[]
valid_all=[]
for _, (train_index, test_index) in enumerate(kf.split(X)):
  x_train,x_test=X[train_index],X[test_index]
  y_train,y_test=y[train_index],y[test_index]
  sc=StandardScaler()
  x_train=sc.fit_transform(x_train)
  x_test=sc.transform(x_test)
  # clf = LogisticRegression(random_state=0).fit(x_train,y_train)
  clf=MLPClassifier(random_state=0,hidden_layer_sizes=(10,3),activation='identity').fit(x_train,y_train)
  cf_all=[]
  ft_type=['ct']*30
  n_ft=30
  start=time.time()
  for k in range(len(x_test)):
    i=x_test[k]
    target=1-y_test[k]
    cls0_var,cls1_var,centroids=get_anchors(x_train,y_train)
    vars,obj_var=create_obj_var2(i,n_ft,ft_type,[cls0_var,cls1_var],centroids,target)
    # vars,obj_var=create_obj_var(i,n_ft,ft_type)
    cf=[]
    target=0.001 if target==0 else 0.999
    vals=optimize(vars,obj_var,clf,i,target)
    for j in vals:
      cf.append(j.value)
    cf_all.append(cf)

  elapsed=time.time()-start


  l1_dist_all.append(l1_dist(cf_all,x_test))
  maha_dist_all.append(maha_dist(cf_all,x_test,np.linalg.pinv(np.cov(x_test.T))))
  valid_all.append(check_validity(cf_all,y_test))
  time_all.append(elapsed/len(x_test))


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
 36750| 2.63e-02  1.59e-07  3.04e-03  9.52e+00  1.00e-01  7.38e-01 
 37000| 2.63e-02  1.09e-07  3.04e-03  9.52e+00  1.00e-01  7.41e-01 
 37250| 2.63e-02  2.12e-07  3.04e-03  9.52e+00  1.00e-01  7.45e-01 
 37500| 2.63e-02  1.05e-07  3.04e-03  9.52e+00  1.00e-01  7.50e-01 
 37750| 2.63e-02  1.65e-07  3.04e-03  9.52e+00  1.00e-01  7.54e-01 
 38000| 2.63e-02  1.26e-07  3.04e-03  9.52e+00  1.00e-01  7.58e-01 
 38250| 2.63e-02  1.08e-07  3.04e-03  9.52e+00  1.00e-01  7.62e-01 
 38500| 7.05e+04  3.10e+02  1.90e+01 -6.91e-02  1.00e-01  7.65e-01 
 38750| 2.63e-02  9.63e-08  3.05e-03  9.52e+00  1.00e-01  7.69e-01 
 39000| 2.63e-02  1.52e-07  3.05e-03  9.52e+00  1.00e-01  7.76e-01 
 39250| 2.63e-02  1.20e-07  3.04e-03  9.52e+00  1.00e-01  7.81e-01 
 39500| 2.63e-02  1.25e-07  3.05e-03  9.52e+00  1.00e-01  7.85e-01 
 39750| 2.63e-02  1.42e-07  3.05e-03  9.52e+00  1.00e-01  7.91e-01 
 40000| 2.63e-02  1.15e-07  3.05e-03  9.52e+00  1.00e-01  7.98e-0

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
(CVXPY) Apr 02 12:48:13 PM: Compiling problem (target solver=SCS).
(CVXPY) Apr 02 12:48:13 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCS
(CVXPY) Apr 02 12:48:13 PM: Applying reduction Dcp2Cone
(CVXPY) Apr 02 12:48:13 PM: Applying reduction CvxAttr2Constr
(CVXPY) Apr 02 12:48:13 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Apr 02 12:48:14 PM: Applying reduction SCS
(CVXPY) Apr 02 12:48:14 PM: Finished problem compilation (took 7.699e-01 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Apr 02 12:48:14 PM: Invoking solver SCS  to obtain a solution.
------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford Univers

/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


...

                                     CVXPY                                     
                                     v1.3.3                                    
(CVXPY) Apr 02 12:49:56 PM: Your problem has 30 variables, 1 constraints, and 0 parameters.
(CVXPY) Apr 02 12:49:56 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 02 12:49:56 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 02 12:49:56 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 02 12:49:56 PM: Compiling problem (target solver=SCS).
(CVXPY) Apr 02 12:49:56 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> 

/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
 11000| 7.13e-01  3.85e-03  6.73e-02  7.55e+00  1.20e-01  3.00e-01 
 11250| 6.15e-01  9.25e-03  7.69e-02  7.54e+00  1.20e-01  3.05e-01 
 11500| 1.47e-01  2.51e-03  2.31e-02  7.59e+00  1.20e-01  3.09e-01 
 11750| 6.64e+00  1.30e+00  3.23e-01  7.67e+00  1.20e-01  3.14e-01 
 12000| 8.14e-01  3.05e-03  1.11e-01  7.52e+00  1.20e-01  3.18e-01 
 12250| 3.85e-01  8.97e-03  6.71e-02  7.54e+00  1.20e-01  3.22e-01 
 12500| 1.12e-01  1.51e-03  1.60e-02  7.58e+00  1.20e-01  3.27e-01 
 12750| 6.72e-01  1.23e-03  9.89e-02  7.62e+00  1.20e-01  3.31e-01 
 13000| 9.75e-01  6.19e-03  4.98e-02  7.54e+00  1.20e-01  3.36e-01 
 13250| 3.12e-01  1.76e-04  3.86e-02  7.56e+00  1.20e-01  3.40e-01 
 13500| 3.85e-01  1.87e-03  8.32e-03  7.57e+00  1.20e-01  3.44e-01 
 13750| 7.94e-01  1.05e-02  2.85e-02  7.54e+00  1.20e-01  3.50e-01 
 14000| 1.77e-01  4.04e-03  7.98e-03  7.57e+00  1.20e-01  3.55e-01 
 14250| 3.22e-01  2.72e-03  2.84e-02  7.60e+00  1.20e-01  3.60e-0

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
------------------------------------------------------------------
objective = 7.050660
------------------------------------------------------------------
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Apr 02 12:58:06 PM: Problem status: optimal
(CVXPY) Apr 02 12:58:06 PM: Optimal value: 7.051e+00
(CVXPY) Apr 02 12:58:06 PM: Compilation took 7.359e-01 seconds
(CVXPY) Apr 02 12:58:06 PM: Solver (including time spent in interface) took 1.627e-01 seconds
status: optimal
                                     CVXPY                                     
                                     v1.3.3                                    
(CVXPY) Apr 02 12:58:07 PM: Your problem has 30 variables, 1 constraints, and 0 parameters.
(CVXPY) Apr 02 12:58:07 

In [ ]:
print('avg l1 {}'.format(np.mean(l1_dist_all)))
print('avg maha {}'.format(np.mean(maha_dist_all)))
print('avg valid {}'.format(np.mean(valid_all)))
print('avg time {}'.format(np.mean(time_all)))

avg l1 29.81314449727922
avg maha 41.92018221345256
avg valid 0.6276665114112715
avg time 1.6943380481355441


In [ ]:
len(l1_dist_all)

5

In [ ]:
acc_all=[]
from sklearn.metrics import accuracy_score
for _, (train_index, test_index) in enumerate(kf.split(X)):
  x_train,x_test=X[train_index],X[test_index]
  y_train,y_test=y[train_index],y[test_index]
  sc=StandardScaler()
  x_train=sc.fit_transform(x_train)
  x_test=sc.transform(x_test)
  # clf = LogisticRegression(random_state=0).fit(x_train,y_train)
  clf=MLPClassifier(random_state=0,hidden_layer_sizes=(10,3),activation='identity').fit(x_train,y_train)
  acc_all.append(accuracy_score(y_test,clf.predict(x_test)))


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
print('avg acc {}'.format(np.mean(acc_all)))

avg acc 0.9736531594472908
